# Running Tune experiments with ZOOpt

In this tutorial we introduce ZOOpt, while running a simple Ray Tune experiment. Tune’s Search Algorithms integrate with ZOOpt and, as a result, allow you to seamlessly scale up a ZOOpt optimization process - without sacrificing performance.

Zeroth-order optimization (ZOOpt) does not rely on the gradient of the objective function, but instead, learns from samples of the search space. It is suitable for optimizing functions that are nondifferentiable, with many local minima, or even unknown but only testable. Therefore, zeroth-order optimization is commonly referred to as "derivative-free optimization" and "black-box optimization". In this example we minimize a simple objective to briefly demonstrate the usage of ZOOpt with Ray Tune via `ZOOptSearch`. It's useful to keep in mind that despite the emphasis on machine learning experiments, Ray Tune optimizes any implicit or explicit objective. Here we assume `zoopt==0.4.1` library is installed. To learn more, please refer to the [ZOOpt website](https://github.com/polixir/ZOOpt).

In [1]:
# !pip install ray[tune]
!pip install zoopt==0.4.1

Click below to see all the imports we need for this example.
You can also launch directly into a Binder instance to run this notebook yourself.
Just click on the rocket symbol at the top of the navigation.

In [2]:
import time

import ray
from ray import tune
from ray.air import session
from ray.tune.search.zoopt import ZOOptSearch
from zoopt import ValueType

Let's start by defining a simple evaluation function.
We artificially sleep for a bit (`0.1` seconds) to simulate a long-running ML experiment.
This setup assumes that we're running multiple `step`s of an experiment and try to tune two hyperparameters,
namely `width` and `height`, and `activation`.

In [3]:
def evaluate(step, width, height):
    time.sleep(0.1)
    return (0.1 + width * step / 100) ** (-1) + height * 0.1

Next, our ``objective`` function takes a Tune ``config``, evaluates the `score` of your experiment in a training loop,
and uses `session.report` to report the `score` back to Tune.

In [4]:
def objective(config):
    for step in range(config["steps"]):
        score = evaluate(step, config["width"], config["height"])
        session.report({"iterations": step, "mean_loss": score})

In [5]:
ray.init(configure_logging=False)

Python version:,3.7.7
Ray version:,3.0.0.dev0
Dashboard:,http://127.0.0.1:8266


Next we define a search space. The critical assumption is that the optimal hyperparameters live within this space. Yet, if the space is very large, then those hyperparameters may be difficult to find in a short amount of time.

In [6]:
search_config = {
    "steps": 100,
    "width": tune.randint(0, 10),
    "height": tune.quniform(-10, 10, 1e-2),
    "activation": tune.choice(["relu, tanh"])
}

The number of samples is the number of hyperparameter combinations that will be tried out. This Tune run is set to `1000` samples.
(you can decrease this if it takes too long on your machine).

In [7]:
num_samples = 1000

In [8]:
# If 1000 samples take too long, you can reduce this number.
# We override this number here for our smoke tests.
num_samples = 10

Next we define the search algorithm built from `ZOOptSearch`, constrained to a maximum of `8` concurrent trials via ZOOpt's internal `"parallel_num"`.

In [9]:
zoopt_config = {
    "parallel_num": 8
}
algo = ZOOptSearch(
    algo="Asracos",  # only supports ASRacos currently
    budget=num_samples,
    **zoopt_config,
)

Finally, we run the experiment to `"min"`imize the "mean_loss" of the `objective` by searching `search_config` via `algo`, `num_samples` times. This previous sentence is fully characterizes the search problem we aim to solve. With this in mind, notice how efficient it is to execute `tuner.fit()`.

In [10]:
tuner = tune.Tuner(
    objective,
    tune_config=tune.TuneConfig(
        metric="mean_loss",
        mode="min",
        search_alg=algo,
        num_samples=num_samples,
    ),
    param_space=search_config,
)
results = tuner.fit()

Function checkpointing is disabled. This may result in unexpected behavior when using checkpointing features or certain schedulers. To enable, set the train function arguments to be `func(config, checkpoint_dir=None)`.


Trial name,status,loc,activation,height,width,loss,iter,total time (s),iterations,neg_mean_loss
objective_8c72f588,TERMINATED,127.0.0.1:47662,"relu, tanh",-3.94,0,9.606,100,10.9102,99,-9.606
objective_8e2f11ae,TERMINATED,127.0.0.1:47667,"relu, tanh",-0.68,6,0.0975629,100,10.7479,99,-0.0975629
objective_8e30a596,TERMINATED,127.0.0.1:47668,"relu, tanh",-5.84,0,9.416,100,10.7724,99,-9.416
objective_8e32324e,TERMINATED,127.0.0.1:47669,"relu, tanh",-2.15,3,0.110733,100,10.7684,99,-0.110733
objective_963faf2a,TERMINATED,127.0.0.1:47689,"relu, tanh",-5.64,6,-0.398437,100,10.8267,99,0.398437
objective_96417df0,TERMINATED,127.0.0.1:47690,"relu, tanh",2.84,6,0.449563,100,10.821,99,-0.449563
objective_96435da0,TERMINATED,127.0.0.1:47691,"relu, tanh",2.6,6,0.425563,100,10.7694,99,-0.425563


Result for objective_8c72f588:
  date: 2022-07-22_15-35-38
  done: false
  experiment_id: 3eb9bcef55e341b0970abd6c1f97eda7
  hostname: Kais-MacBook-Pro.local
  iterations: 0
  iterations_since_restore: 1
  mean_loss: 9.606
  neg_mean_loss: -9.606
  node_ip: 127.0.0.1
  pid: 47662
  time_since_restore: 0.10410094261169434
  time_this_iter_s: 0.10410094261169434
  time_total_s: 0.10410094261169434
  timestamp: 1658500538
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: 8c72f588
  warmup_time: 0.003092050552368164
  
Result for objective_8e30a596:
  date: 2022-07-22_15-35-41
  done: false
  experiment_id: d58453075b71453ab615e10ae9713072
  hostname: Kais-MacBook-Pro.local
  iterations: 0
  iterations_since_restore: 1
  mean_loss: 9.416
  neg_mean_loss: -9.416
  node_ip: 127.0.0.1
  pid: 47668
  time_since_restore: 0.1051950454711914
  time_this_iter_s: 0.1051950454711914
  time_total_s: 0.1051950454711914
  timestamp: 1658500541
  timesteps_since_restore: 0
  training_iter

Result for objective_96417df0:
  date: 2022-07-22_15-35-54
  done: false
  experiment_id: e98b245717b4423d8917589cf5d42088
  hostname: Kais-MacBook-Pro.local
  iterations: 0
  iterations_since_restore: 1
  mean_loss: 10.284
  neg_mean_loss: -10.284
  node_ip: 127.0.0.1
  pid: 47690
  time_since_restore: 0.10359907150268555
  time_this_iter_s: 0.10359907150268555
  time_total_s: 0.10359907150268555
  timestamp: 1658500554
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: 96417df0
  warmup_time: 0.003325939178466797
  
Result for objective_96435da0:
  date: 2022-07-22_15-35-54
  done: false
  experiment_id: 8680a80a099c452dadd3be44d3457ca5
  hostname: Kais-MacBook-Pro.local
  iterations: 0
  iterations_since_restore: 1
  mean_loss: 10.26
  neg_mean_loss: -10.26
  node_ip: 127.0.0.1
  pid: 47691
  time_since_restore: 0.10206389427185059
  time_this_iter_s: 0.10206389427185059
  time_total_s: 0.10206389427185059
  timestamp: 1658500554
  timesteps_since_restore: 0
  training

Here are the hyperparamters found to minimize the mean loss of the defined objective.

In [11]:
print("Best hyperparameters found were: ", results.get_best_result().config)

Best hyperparameters found were:  {'steps': 100, 'width': 6, 'height': -5.64, 'activation': 'relu, tanh'}


## Optional: passing the parameter space into the search algorithm

We can also pass the parameter space ourselves in the following formats: 
- continuous dimensions: (continuous, search_range, precision)
- discrete dimensions: (discrete, search_range, has_order)
- grid dimensions: (grid, grid_list)

In [12]:
space = {
    "height": (ValueType.CONTINUOUS, [-10, 10], 1e-2),
    "width": (ValueType.DISCRETE, [0, 10], True),
    "layers": (ValueType.GRID, [4, 8, 16])
}

ZOOpt again handles constraining the amount of concurrent trials with `"parallel_num"`.

In [13]:
zoopt_search_config = {
    "parallel_num": 8,
    "metric": "mean_loss",
    "mode": "min"
}
algo = ZOOptSearch(
    algo="Asracos",
    budget=num_samples,
    dim_dict=space,
    **zoopt_search_config
)

This time we pass only `"steps"` and `"activation"` to the Tune `config` because `"height"` and `"width"` have been passed into `ZOOptSearch` to create the `search_algo`. 
Again, we run the experiment to `"min"`imize the "mean_loss" of the `objective` by searching `search_config` via `algo`, `num_samples` times.

In [14]:
tuner = tune.Tuner(
    objective,
    tune_config=tune.TuneConfig(
        metric="mean_loss",
        mode="min",
        search_alg=algo,
        num_samples=num_samples,
    ),
    param_space={"steps": 100},
)
results = tuner.fit()

Trial name,status,loc,height,layers,width,loss,iter,total time (s),iterations,neg_mean_loss
objective_9e64c92e,TERMINATED,127.0.0.1:47713,-8.08,16,0,9.192,100,10.7118,99,-9.192
objective_9ff31930,TERMINATED,127.0.0.1:47718,0.38,16,7,0.180248,100,10.7315,99,-0.180248
objective_9ff47d0c,TERMINATED,127.0.0.1:47719,5.09,4,10,0.609,100,10.7924,99,-0.609
objective_9ff5c2b6,TERMINATED,127.0.0.1:47720,5.26,16,1,1.44343,100,10.7868,99,-1.44343
objective_a7f414d6,TERMINATED,127.0.0.1:47737,0.38,4,7,0.180248,100,10.7232,99,-0.180248
objective_a7f5c682,TERMINATED,127.0.0.1:47738,-2.38,16,7,-0.0957525,100,10.7337,99,0.0957525
objective_a7f7c162,TERMINATED,127.0.0.1:47739,0.38,8,7,0.180248,100,10.7452,99,-0.180248
objective_a7f96fda,TERMINATED,127.0.0.1:47740,0.38,16,4,0.284305,100,10.7079,99,-0.284305
objective_a7fb1844,TERMINATED,127.0.0.1:47741,0.38,8,7,0.180248,100,10.7157,99,-0.180248
objective_a7fcf02e,TERMINATED,127.0.0.1:47742,-8.88,16,7,-0.745752,100,10.7305,99,0.745752


Result for objective_9e64c92e:
  date: 2022-07-22_15-36-08
  done: false
  experiment_id: fafb7d88360d408286e616de3dcd4407
  hostname: Kais-MacBook-Pro.local
  iterations: 0
  iterations_since_restore: 1
  mean_loss: 9.192
  neg_mean_loss: -9.192
  node_ip: 127.0.0.1
  pid: 47713
  time_since_restore: 0.1042020320892334
  time_this_iter_s: 0.1042020320892334
  time_total_s: 0.1042020320892334
  timestamp: 1658500568
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: 9e64c92e
  warmup_time: 0.0030999183654785156
  
Result for objective_9ff47d0c:
  date: 2022-07-22_15-36-11
  done: false
  experiment_id: de33a45d0c344d3aa344d8cff20b6c37
  hostname: Kais-MacBook-Pro.local
  iterations: 0
  iterations_since_restore: 1
  mean_loss: 10.509
  neg_mean_loss: -10.509
  node_ip: 127.0.0.1
  pid: 47719
  time_since_restore: 0.10373878479003906
  time_this_iter_s: 0.10373878479003906
  time_total_s: 0.10373878479003906
  timestamp: 1658500571
  timesteps_since_restore: 0
  training_i

Result for objective_a7fb1844:
  date: 2022-07-22_15-36-24
  done: false
  experiment_id: 668080a52a3f4c4fad2258177b1fb755
  hostname: Kais-MacBook-Pro.local
  iterations: 0
  iterations_since_restore: 1
  mean_loss: 10.038
  neg_mean_loss: -10.038
  node_ip: 127.0.0.1
  pid: 47741
  time_since_restore: 0.10408973693847656
  time_this_iter_s: 0.10408973693847656
  time_total_s: 0.10408973693847656
  timestamp: 1658500584
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: a7fb1844
  warmup_time: 0.004377126693725586
  
Result for objective_a7f5c682:
  date: 2022-07-22_15-36-24
  done: false
  experiment_id: 4449a3ef690c4b76a0cfbf7c4de7df43
  hostname: Kais-MacBook-Pro.local
  iterations: 0
  iterations_since_restore: 1
  mean_loss: 9.762
  neg_mean_loss: -9.762
  node_ip: 127.0.0.1
  pid: 47738
  time_since_restore: 0.10507631301879883
  time_this_iter_s: 0.10507631301879883
  time_total_s: 0.10507631301879883
  timestamp: 1658500584
  timesteps_since_restore: 0
  training

Result for objective_a7fb1844:
  date: 2022-07-22_15-36-35
  done: true
  experiment_id: 668080a52a3f4c4fad2258177b1fb755
  experiment_tag: 9_height=0.3800,layers=8,steps=100,width=7
  hostname: Kais-MacBook-Pro.local
  iterations: 99
  iterations_since_restore: 100
  mean_loss: 0.18024751066856332
  neg_mean_loss: -0.18024751066856332
  node_ip: 127.0.0.1
  pid: 47741
  time_since_restore: 10.715673923492432
  time_this_iter_s: 0.10643696784973145
  time_total_s: 10.715673923492432
  timestamp: 1658500595
  timesteps_since_restore: 0
  training_iteration: 100
  trial_id: a7fb1844
  warmup_time: 0.004377126693725586
  
Result for objective_a7f414d6:
  date: 2022-07-22_15-36-35
  done: true
  experiment_id: 20575f602c234dd7a167a93ff353299b
  experiment_tag: 5_height=0.3800,layers=4,steps=100,width=7
  hostname: Kais-MacBook-Pro.local
  iterations: 99
  iterations_since_restore: 100
  mean_loss: 0.18024751066856332
  neg_mean_loss: -0.18024751066856332
  node_ip: 127.0.0.1
  pid: 47737
 

Here are the hyperparamters found to minimize the mean loss of the defined objective.

In [15]:
print("Best hyperparameters found were: ", results.get_best_result().config)

Best hyperparameters found were:  {'steps': 100, 'height': -8.88, 'width': 7, 'layers': 16}


In [16]:
ray.shutdown()